In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import datetime as dt

DATAPATH = './models/'
# GPU training
DEVICE = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
#DEVICE = torch.device('cpu')
print(f"Training on device {DEVICE}.")
torch.cuda.memory_allocated()

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training on device cuda.


0

In [2]:
from cocoBox import load_coco_dataset

BATCH_SIZE = 32
IMG_SIZE = 300
# Select image categories with which to train SSD
categories = ('horse', 'bird', 'cat', 'dog', 'person', 'car')
train_dataloader, valid_dataloader = load_coco_dataset(batch_size=BATCH_SIZE,
                                                       size=1000,
                                                       dim=IMG_SIZE,
                                                       cats=categories,
                                                       fetch_type='union')

loading annotations into memory...
Done (t=19.33s)
creating index...
index created!
Categories: {19: 0, 16: 1, 17: 2, 18: 3, 1: 4, 3: 5}
Dataset size: 1000
loading annotations into memory...
Done (t=13.94s)
creating index...
index created!
Categories: {19: 0, 16: 1, 17: 2, 18: 3, 1: 4, 3: 5}
Dataset size: 1000
Dataset with augmentation size: 2000


In [3]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, valid_loader, max_patience=10):
    best_valid_loss = float('Inf')
    print('{} starting training'.format(dt.datetime.now()))
    for epoch in range(1, n_epochs + 1):
        train_loss = 0.0
        for images, targets in train_loader:
            images = torch.cat(list(map(lambda x: x.unsqueeze(0), images))).to(device=DEVICE)
            boxes = tuple(map(lambda x: torch.tensor(x['boxes']).to(device=DEVICE), targets))
            labels = tuple(map(lambda x: torch.tensor(x['labels']).to(device=DEVICE), targets))
            coords, conf = model(images)
            loss = loss_fn(coords, conf, boxes, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        torch.cuda.empty_cache()
        # Validation
        with torch.no_grad():
            valid_loss = 0.0
            for images, targets in train_loader:
                images = torch.cat(list(map(lambda x: x.unsqueeze(0), images))).to(device=DEVICE)
                boxes = tuple(map(lambda x: torch.tensor(x['boxes']).to(device=DEVICE), targets))
                labels = tuple(map(lambda x: torch.tensor(x['labels']).to(device=DEVICE), targets))
                coords, conf = model(images)
                loss = loss_fn(coords, conf, boxes, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                valid_loss += loss.item()
        torch.cuda.empty_cache()
        # Verbose training
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Train {:.5f}, Valid {:.5f}'.format(dt.datetime.now(),
                                                                      epoch,
                                                                      train_loss,
                                                                      valid_loss))
        if valid_loss < best_valid_loss:
            torch.save(model.state_dict(), DATAPATH + 'ssd.pt')
            print(f'Saving {epoch}-th for {valid_loss = :2.5f}')
            best_valid_loss = valid_loss
            patience = max_patience
        elif patience == 0:
            return
        else:
            patience = patience - 1


In [6]:
#import importlib
#import SSD
#importlib.reload(SSD)
#import utils
#importlib.reload(utils)
from SSD import SSD
#import SSDLoss
#importlib.reload(SSDLoss)
from SSDLoss import SSDLoss

ssd = SSD(class_num=len(categories)).to(device=DEVICE)
adam = torch.optim.Adam(ssd.parameters(), lr=1e-3)
training_loop(n_epochs=1000,
              optimizer=adam,
              loss_fn=SSDLoss().to(device=DEVICE),
              model=ssd,
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)

2023-01-18 20:32:36.594390 starting training


OutOfMemoryError: CUDA out of memory. Tried to allocate 704.00 MiB (GPU 0; 14.61 GiB total capacity; 12.73 GiB already allocated; 675.19 MiB free; 12.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches
# Load best models
ssd.load_state_dict(torch.load(DATAPATH + 'ssd.pt', map_location=DEVICE))
plt.figure(figsize=(12, 60))
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
mean = [-1 * x / y for x, y in zip(mean, std)]
std = [1 / x for x in std]
unnormalize = torchvision.transforms.Normalize(mean=mean, std=std)
for imgs, targets in valid_dataloader:
    if len(targets[0]['boxes']) < 2:
        continue
    fig, ax = plt.subplots()
    img = imgs[0]
    target = targets[0]
    ax.imshow(unnormalize(img).permute(1, 2, 0), vmin=0, vmax=1)
    # TODO: add labelling of rectangle box
    for idx, box in enumerate(target['boxes']):
        x = box[0] * img.shape[2]
        y = box[1] * img.shape[1]
        width = box[2] * img.shape[2]
        height = box[3] * img.shape[1]
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor="b", fill=False)
        ax.add_patch(rect)
        ax.text(x=x, y=y, s=categories[target['label'][idx]])
    offs, conf = ssd.predict(imgs, targets)
    # TODO: add labelling of rectangle box
    for idx, box in enumerate(offs):
        x = box[0] * img.shape[2]
        y = box[1] * img.shape[1]
        width = box[2] * img.shape[2]
        height = box[3] * img.shape[1]
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor="r", fill=False)
        ax.add_patch(rect)
        ax.text(x=x, y=y, s=categories[np.argmax(conf[idx])])
    plt.show()